# Документация к реализации алгоритма PPO

## Содержание
1. [Введение](#введение)
2. [Архитектура](#архитектура)
3. [Основные компоненты](#основные-компоненты)
4. [Рабочий процесс (Workflow)](#рабочий-процесс-workflow)
5. [Детали реализации](#детали-реализации)
6. [Гиперпараметры](#гиперпараметры)
7. [Использование](#использование)

## Введение

Данный код представляет собой реализацию алгоритма Proximal Policy Optimization (PPO) для задач обучения с подкреплением. PPO - это метод оптимизации стратегии, предложенный OpenAI, который обеспечивает стабильное обучение, предотвращая слишком большие обновления политики.

Реализация включает в себя:
- Класс нейронной сети прямого распространения (`FeedForwardNN`)
- Класс PPO для реализации алгоритма
- Функционал для обучения на средах OpenAI Gym

## Архитектура

Система имеет следующую общую архитектуру:

```
                       +-------------------+
                       |     Среда Gym     |
                       | (напр. Pendulum)  |
                       +--------+----------+
                                |
                                | Наблюдения, Награды
                                v
+---------------+    +-------------------------+    +-----------------+
| Актор-нейросеть|<-->|      Алгоритм PPO      |<-->| Критик-нейросеть|
+---------------+    +-------------------------+    +-----------------+
                                |
                                | Действия
                                v
                       +-------------------+
                       |     Среда Gym     |
                       +-------------------+
```

## Основные компоненты

### 1. Класс FeedForwardNN

Стандартная нейронная сеть прямого распространения с архитектурой:
- Входной слой (размерность зависит от пространства наблюдений среды)
- Два скрытых слоя по 64 нейрона каждый с функцией активации ReLU
- Выходной слой (размерность зависит от пространства действий среды для актора или 1 для критика)

Сеть используется для реализации как актора (выбор действий), так и критика (оценка ценности состояний).

### 2. Класс PPO

Основной класс, реализующий алгоритм PPO. Включает в себя:
- Инициализацию актора и критика
- Функции для сбора данных в среде (rollout)
- Функции для вычисления Rewards-To-Go и преимуществ
- Функции для обновления сетей
- Логгирование процесса обучения

## Рабочий процесс (Workflow)

### Этап 1: Инициализация
1. Создание среды обучения с использованием OpenAI Gym
2. Инициализация актор-сети и критик-сети с заданной архитектурой
3. Настройка гиперпараметров алгоритма
4. Инициализация оптимизаторов и логгера

### Этап 2: Сбор данных (Rollout)
1. Взаимодействие с средой до тех пор, пока не будет собрано указанное количество шагов (`timesteps_per_batch`)
2. Для каждого шага:
   - Получение текущего наблюдения
   - Выбор действия на основе текущей политики (актор-сеть)
   - Выполнение действия в среде
   - Сохранение наблюдения, действия, вероятности действия и награды
3. Вычисление Rewards-To-Go для каждого шага (дисконтированная сумма будущих наград)

### Этап 3: Обновление моделей
1. Вычисление текущих оценок значений (Value) для всех наблюдений
2. Вычисление преимуществ (Advantage) для каждого шага
3. Нормализация преимуществ для стабильности обучения
4. Выполнение нескольких эпох обучения (`n_updates_per_iteration`):
   - Вычисление новых логарифмов вероятностей действий и оценок значений
   - Вычисление отношения новых и старых вероятностей (ratios)
   - Вычисление суррогатных потерь с клиппингом для предотвращения слишком больших обновлений
   - Обновление весов актор-сети и критик-сети с использованием обратного распространения ошибки

### Этап 4: Логгирование и сохранение
1. Вывод информации о процессе обучения (средняя длина эпизода, средний возврат, потери)
2. Периодическое сохранение моделей

### Этап 5: Повторение
Повторение этапов 2-4 до достижения указанного количества шагов (`total_timesteps`)

## Детали реализации

### Выбор действий
Актор-сеть выдает среднее значение нормального распределения для каждого измерения пространства действий. Действия выбираются путем сэмплирования из многомерного нормального распределения с диагональной ковариационной матрицей.

### Вычисление Rewards-To-Go
Для каждого временного шага вычисляется сумма текущей и будущих дисконтированных наград:

$$
RTG_t = r_t + gamma * r_{t+1} + gamma^2 * r_{t+2} + ...
$$

### Вычисление преимуществ
Преимущества вычисляются как разница между реальным Reward-To-Go и предсказанным значением Value:

$$
A_t = RTG_t - V(s_t)
$$

### Обновление политики
Используется клиппированная функция потерь для обеспечения консервативных обновлений:

$$
L = min(r_t * A_t, clip(r_t, 1-epsilon, 1+epsilon) * A_t)
$$

где $r_t$ - отношение новой и старой вероятностей действия, а $epsilon$ - параметр клиппинга.

## Гиперпараметры

| Параметр | Значение по умолчанию | Описание |
|----------|----------------------|-----------|
| `timesteps_per_batch` | 4800 | Количество шагов в одном батче |
| `max_timesteps_per_episode` | 1600 | Максимальное количество шагов в одном эпизоде |
| `n_updates_per_iteration` | 5 | Количество эпох обновления на каждой итерации |
| `lr` | 0.005 | Скорость обучения |
| `gamma` | 0.95 | Коэффициент дисконтирования |
| `clip` | 0.2 | Параметр клиппинга для PPO |
| `render` | True | Включение визуализации |
| `render_every_i` | 10 | Частота визуализации (каждые N итераций) |
| `save_freq` | 10 | Частота сохранения модели (каждые N итераций) |

## Использование

```python
# Создаем среду
env = gym.make('Pendulum-v1')

# Инициализируем PPO с пользовательскими гиперпараметрами
hyperparameters = {
    'timesteps_per_batch': 2048,
    'max_timesteps_per_episode': 200,
    'gamma': 0.99,
    'n_updates_per_iteration': 10,
    'lr': 3e-4,
    'clip': 0.2,
    'render': True,
    'render_every_i': 10
}

# Создаем экземпляр PPO
ppo = PPO(FeedForwardNN, env, **hyperparameters)

# Запускаем обучение
ppo.learn(total_timesteps=10000)
```

In [14]:
# Импорт стандартных библиотек
import os
import time
from typing import Any, Dict, List, Optional, Tuple

# Импорт сторонних библиотек
import gymnasium as gym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
from torch.optim import Adam

# Импорт библиотек для визуализации
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from IPython.display import clear_output, display
from matplotlib.figure import Figure
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm

# Настройка стиля визуализации
plt.style.use('ggplot')
sns.set_theme(style="darkgrid")

# Создание директории для сохранения графиков
os.makedirs('visualizations', exist_ok=True)

In [15]:
class FeedForwardNN(nn.Module):
    """
    Description:
    ---------------
        Стандартная нейронная сеть с архитектурой in_dim-64-64-out_dim.

    Args:
    ---------------
        in_dim (int): Размерность входных данных.
        out_dim (int): Размерность выходных данных.

    Returns:
    ---------------
        None
    """

    def __init__(self, in_dim: int, out_dim: int) -> None:
        """
        Description:
        ---------------

            Инициализация сети и настройка слоев.

        Args:
        ---------------
            in_dim (int): Размерность входных данных.
            out_dim (int): Размерность выходных данных.

        Returns:
        ---------------
            None
        """
        super(FeedForwardNN, self).__init__()

        # Определение слоев нейронной сети
        self.layer1 = nn.Linear(in_dim, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, out_dim)


    def forward(self, obs: np.ndarray) -> torch.Tensor:
        """
        Description:
        ---------------
            Выполняет прямой проход по нейронной сети.

        Args:
        ---------------
            obs (np.ndarray): Входные данные для обработки.

        Returns:
        ---------------
            torch.Tensor: Выходные данные после обработки.

        Raises:
        ---------------
            TypeError: Если входные данные не являются экземпляром np.ndarray или torch.Tensor.

        Examples:
        ---------------
            >>> model = FeedForwardNN(10, 2)
            >>> output = model.forward(np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))
        """
        # Преобразование входных данных в тензор, если это необходимо
        if isinstance(obs, np.ndarray):
            obs = torch.tensor(obs, dtype=torch.float)

        # Прямой проход по слоям сети с сохранением активаций для визуализации
        activation1 = F.relu(self.layer1(obs))
        self.activations['layer1'] = activation1.detach().numpy() if isinstance(activation1, torch.Tensor) else activation1

        activation2 = F.relu(self.layer2(activation1))
        self.activations['layer2'] = activation2.detach().numpy() if isinstance(activation2, torch.Tensor) else activation2

        output = self.layer3(activation2)

        # Сохраняем веса для визуализации
        self.weights['layer1'] = self.layer1.weight.detach().numpy()
        self.weights['layer2'] = self.layer2.weight.detach().numpy()
        self.weights['layer3'] = self.layer3.weight.detach().numpy()

        return output

class PPO:
    """
    Description:
    ---------------
        Класс PPO, используемый в качестве модели в main.py.

    Args:
    ---------------
        policy_class: Класс политики для использования в актор/критик сетях.
        env: Среда для обучения.
        hyperparameters: Дополнительные параметры гиперпараметров.

    Returns:
    ---------------
        None
    """

    def __init__(self, policy_class: Any, env: gym.Env, **hyperparameters: Any) -> None:
        """
        Description:
        ---------------
            Инициализация модели PPO, включая гиперпараметры.

        Args:
        ---------------
            policy_class: Класс политики для использования в актор/критик сетях.
            env: Среда для обучения.
            hyperparameters: Дополнительные параметры гиперпараметров.

        Returns:
        ---------------
            None
        """
        # Проверка совместимости среды
        assert isinstance(env.observation_space, gym.spaces.Box)
        assert isinstance(env.action_space, gym.spaces.Box)

        # Инициализация гиперпараметров для обучения с PPO
        self._init_hyperparameters(hyperparameters)

        # Извлечение информации о среде
        self.env = env
        self.obs_dim = env.observation_space.shape[0]
        self.act_dim = env.action_space.shape[0]

        # Инициализация актор и критик сетей
        self.actor = policy_class(self.obs_dim, self.act_dim)
        self.critic = policy_class(self.obs_dim, 1)

        # Инициализация оптимизаторов для актор и критик сетей
        self.actor_optim = Adam(self.actor.parameters(), lr=self.lr)
        self.critic_optim = Adam(self.critic.parameters(), lr=self.lr)

        # Инициализация ковариационной матрицы для запроса действий у актора
        self.cov_var = torch.full(size=(self.act_dim,), fill_value=0.5)
        self.cov_mat = torch.diag(self.cov_var)

        # Логгер для вывода сводок каждой итерации
        self.logger = {
            'delta_t': time.time_ns(),
            't_so_far': 0,          # количество временных шагов на данный момент
            'i_so_far': 0,          # количество итераций на данный момент
            'batch_lens': [],       # длины эпизодов в батче
            'batch_rews': [],       # возвраты эпизодов в батче
            'actor_losses': [],     # потери актор сети в текущей итерации
        }

    def learn(self, total_timesteps: int) -> None:
        """
        Description:
        ---------------
            Обучение актор и критик сетей. Основной алгоритм PPO.

        Args:
        ---------------
            total_timesteps: Общее количество временных шагов для обучения.

        Returns:
        ---------------
            None
        """
        print(f"Обучение... Выполняется {self.max_timesteps_per_episode} временных шагов за эпизод, ", end='')
        print(f"{self.timesteps_per_batch} временных шагов за батч, всего {total_timesteps} временных шагов")

        t_so_far = 0  # Количество временных шагов, симулированных на данный момент
        i_so_far = 0  # Количество выполненных итераций

        while t_so_far < total_timesteps:
            # Сбор батча симуляций
            batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens = self.rollout()

            # Подсчет количества временных шагов, собранных в этом батче
            t_so_far += np.sum(batch_lens)

            # Увеличение количества итераций
            i_so_far += 1

            # Логирование временных шагов и итераций
            self.logger['t_so_far'] = t_so_far
            self.logger['i_so_far'] = i_so_far

            # Вычисление преимущества на k-й итерации
            V, _ = self.evaluate(batch_obs, batch_acts)
            A_k = batch_rtgs - V.detach()

            # Нормализация преимуществ для уменьшения дисперсии
            A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10)

            # Обновление сети в течение нескольких эпох
            for _ in range(self.n_updates_per_iteration):
                # Вычисление V_phi и pi_theta(a_t | s_t)
                V, curr_log_probs = self.evaluate(batch_obs, batch_acts)

                # Вычисление отношения pi_theta(a_t | s_t) / pi_theta_k(a_t | s_t)
                ratios = torch.exp(curr_log_probs - batch_log_probs)

                # Вычисление суррогатных потерь
                surr1 = ratios * A_k
                surr2 = torch.clamp(ratios, 1 - self.clip, 1 + self.clip) * A_k

                # Вычисление потерь актор и критик сетей
                actor_loss = (-torch.min(surr1, surr2)).mean()
                critic_loss = nn.MSELoss()(V, batch_rtgs)

                # Обратное распространение для актор сети
                self.actor_optim.zero_grad()
                actor_loss.backward(retain_graph=True)
                self.actor_optim.step()

                # Обратное распространение для критик сети
                self.critic_optim.zero_grad()
                critic_loss.backward()
                self.critic_optim.step()

                # Логирование потерь актора
                self.logger['actor_losses'].append(actor_loss.detach())

            # Вывод сводки обучения
            self._log_summary()

            # Сохранение модели
            if i_so_far % self.save_freq == 0:
                torch.save(self.actor.state_dict(), './ppo_actor.pth')
                torch.save(self.critic.state_dict(), './ppo_critic.pth')

    def rollout(self) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, List[int]]:
        """
        Description:
        ---------------
            Сбор батча данных из симуляции.

        Args:
        ---------------
            None

        Returns:
        ---------------
            batch_obs: Наблюдения, собранные в этом батче.
            batch_acts: Действия, собранные в этом батче.
            batch_log_probs: Логарифмы вероятностей каждого действия в этом батче.
            batch_rtgs: Rewards-To-Go каждого временного шага в этом батче.
            batch_lens: Длины каждого эпизода в этом батче.
        """
        # Данные батча
        batch_obs = []
        batch_acts = []
        batch_log_probs = []
        batch_rews = []
        batch_rtgs = []
        batch_lens = []

        # Данные эпизода
        ep_rews = []

        t = 0  # Количество временных шагов, выполненных в этом батче

        # Симуляция до достижения указанного количества временных шагов
        while t < self.timesteps_per_batch:
            ep_rews = []  # награды, собранные за эпизод

            # Сброс среды
            obs, _ = self.env.reset()
            done = False

            # Выполнение эпизода
            for ep_t in range(self.max_timesteps_per_episode):
                # Рендеринг среды, если указано
                if self.render and (self.logger['i_so_far'] % self.render_every_i == 0) and len(batch_lens) == 0:
                    self.env.render()

                t += 1  # Увеличение количества временных шагов в этом батче

                # Отслеживание наблюдений в этом батче
                batch_obs.append(obs)

                # Вычисление действия и выполнение шага в среде
                action, log_prob = self.get_action(obs)
                obs, rew, terminated, truncated, _ = self.env.step(action)

                # Объединение terminated и truncated
                done = terminated or truncated

                # Отслеживание наград, действий и логарифмов вероятностей
                ep_rews.append(rew)
                batch_acts.append(action)
                batch_log_probs.append(log_prob)

                # Завершение эпизода, если среда указала на его окончание
                if done:
                    break

            # Отслеживание длины и наград эпизода
            batch_lens.append(ep_t + 1)
            batch_rews.append(ep_rews)

        # Преобразование данных в тензоры
        batch_obs = torch.tensor(batch_obs, dtype=torch.float)
        batch_acts = torch.tensor(batch_acts, dtype=torch.float)
        batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float)
        batch_rtgs = self.compute_rtgs(batch_rews)

        # Логирование возвратов и длины эпизодов в этом батче
        self.logger['batch_rews'] = batch_rews
        self.logger['batch_lens'] = batch_lens

        return batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens

    def compute_rtgs(self, batch_rews: List[List[float]]) -> torch.Tensor:
        """
        Description:
        ---------------
            Вычисление Reward-To-Go каждого временного шага в батче.

        Args:
        ---------------
            batch_rews: Награды в батче.

        Returns:
        ---------------
            batch_rtgs: Rewards-To-Go.
        """
        # Rewards-To-Go для каждого эпизода в батче
        batch_rtgs = []

        # Итерация по каждому эпизоду
        for ep_rews in reversed(batch_rews):
            discounted_reward = 0  # Накопленная дисконтированная награда

            # Итерация по всем наградам в эпизоде
            for rew in reversed(ep_rews):
                discounted_reward = rew + discounted_reward * self.gamma
                batch_rtgs.insert(0, discounted_reward)

        # Преобразование Rewards-To-Go в тензор
        batch_rtgs = torch.tensor(batch_rtgs, dtype=torch.float)

        return batch_rtgs

    def get_action(self, obs: np.ndarray) -> Tuple[np.ndarray, torch.Tensor]:
        """
        Description:
        ---------------
            Запрос действия у актор сети.

        Args:
        ---------------
            obs: Наблюдение на текущем временном шаге.

        Returns:
        ---------------
            action: Действие для выполнения.
            log_prob: Логарифм вероятности выбранного действия.
        """
        # Запрос среднего действия у актор сети
        mean = self.actor(obs)

        # Создание распределения с средним действием
        dist = MultivariateNormal(mean, self.cov_mat)

        # Выбор действия из распределения
        action = dist.sample()

        # Вычисление логарифма вероятности действия
        log_prob = dist.log_prob(action)

        # Возврат выбранного действия и логарифма вероятности
        return action.detach().numpy(), log_prob.detach()

    def evaluate(self, batch_obs: torch.Tensor, batch_acts: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Description:
        ---------------
            Оценка значений каждого наблюдения и логарифмов вероятностей действий.

        Args:
        ---------------
            batch_obs: Наблюдения из последнего собранного батча.
            batch_acts: Действия из последнего собранного батча.

        Returns:
        ---------------
            V: Предсказанные значения batch_obs.
            log_probs: Логарифмы вероятностей действий batch_acts при batch_obs.
        """
        # Запрос значений V у критик сети
        V = self.critic(batch_obs).squeeze()

        # Вычисление логарифмов вероятностей действий
        mean = self.actor(batch_obs)
        dist = MultivariateNormal(mean, self.cov_mat)
        log_probs = dist.log_prob(batch_acts)

        # Возврат значений и логарифмов вероятностей
        return V, log_probs

    def _init_hyperparameters(self, hyperparameters: dict) -> None:
        """
        Description:
        ---------------
            Инициализация значений гиперпараметров.

        Args:
        ---------------
            hyperparameters: Дополнительные параметры гиперпараметров.

        Returns:
        ---------------
            None
        """
        # Инициализация значений гиперпараметров по умолчанию
        self.timesteps_per_batch = 4800
        self.max_timesteps_per_episode = 1600
        self.n_updates_per_iteration = 5
        self.lr = 0.005
        self.gamma = 0.95
        self.clip = 0.2
        self.render = True
        self.render_every_i = 10
        self.save_freq = 10
        self.seed = None

        # Изменение значений по умолчанию на пользовательские
        for param, val in hyperparameters.items():
            setattr(self, param, val)

        # Установка seed, если указано
        if self.seed is not None:
            assert isinstance(self.seed, int)
            torch.manual_seed(self.seed)
            print(f"Seed успешно установлен на {self.seed}")

    def _log_summary(self) -> None:
        """
        Description:
        ---------------
            Вывод сводки обучения.

        Args:
        ---------------
            None

        Returns:
        ---------------
            None
        """
        # Вычисление значений для логирования
        delta_t = self.logger['delta_t']
        self.logger['delta_t'] = time.time_ns()
        delta_t = (self.logger['delta_t'] - delta_t) / 1e9
        delta_t = str(round(delta_t, 2))

        t_so_far = self.logger['t_so_far']
        i_so_far = self.logger['i_so_far']
        avg_ep_lens = np.mean(self.logger['batch_lens'])
        avg_ep_rews = np.mean([np.sum(ep_rews) for ep_rews in self.logger['batch_rews']])
        avg_actor_loss = np.mean([losses.float().mean() for losses in self.logger['actor_losses']])

        # Округление значений для более эстетичного вывода
        avg_ep_lens = str(round(avg_ep_lens, 2))
        avg_ep_rews = str(round(avg_ep_rews, 2))
        avg_actor_loss = str(round(avg_actor_loss, 5))

        # Вывод сводки
        print(flush=True)
        print(f"-------------------- Итерация #{i_so_far} --------------------", flush=True)
        print(f"Средняя длина эпизода: {avg_ep_lens}", flush=True)
        print(f"Средний возврат эпизода: {avg_ep_rews}", flush=True)
        print(f"Средняя потеря: {avg_actor_loss}", flush=True)
        print(f"Временных шагов на данный момент: {t_so_far}", flush=True)
        print(f"Итерация заняла: {delta_t} сек", flush=True)
        print(f"------------------------------------------------------", flush=True)
        print(flush=True)

        # Сброс данных логирования для батча
        self.logger['batch_lens']   = []
        self.logger['batch_rews']   = []
        self.logger['actor_losses'] = []

In [16]:
# Создаем среду
env = gym.make('Pendulum-v1')  # Можно попробовать другие среды Box2D или MuJoCo

# Инициализируем PPO
hyperparameters = {
    'timesteps_per_batch': 2048,
    'max_timesteps_per_episode': 200,
    'gamma': 0.99,
    'n_updates_per_iteration': 10,
    'lr': 3e-4,
    'clip': 0.2,
    'render': True,
    'render_every_i': 10
}

ppo = PPO(FeedForwardNN, env, **hyperparameters)

# Запускаем обучение
ppo.learn(total_timesteps=10000)

Обучение... Выполняется 200 временных шагов за эпизод, 2048 временных шагов за батч, всего 10000 временных шагов


/usr/local/lib/python3.11/dist-packages/gymnasium/envs/classic_control/pendulum.py:178: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("Pendulum-v1", render_mode="rgb_array")
  gym.logger.warn(



-------------------- Итерация #1 --------------------
Средняя длина эпизода: 200.0
Средний возврат эпизода: -1223.48
Средняя потеря: -0.00306
Временных шагов на данный момент: 2200
Итерация заняла: 3.33 сек
------------------------------------------------------


-------------------- Итерация #2 --------------------
Средняя длина эпизода: 200.0
Средний возврат эпизода: -1165.36
Средняя потеря: -0.00077
Временных шагов на данный момент: 4400
Итерация заняла: 3.42 сек
------------------------------------------------------


-------------------- Итерация #3 --------------------
Средняя длина эпизода: 200.0
Средний возврат эпизода: -1221.3
Средняя потеря: -0.00152
Временных шагов на данный момент: 6600
Итерация заняла: 2.96 сек
------------------------------------------------------


-------------------- Итерация #4 --------------------
Средняя длина эпизода: 200.0
Средний возврат эпизода: -1235.85
Средняя потеря: -0.00168
Временных шагов на данный момент: 8800
Итерация заняла: 2.93 сек
-

In [17]:
# Тестирование обученного агента
def test_agent(ppo: Any, env: gym.Env, n_episodes: int = 10) -> None:
    """
    Description:
    ---------------
        Тестирование обученного агента в заданной среде.

    Args:
    ---------------
        ppo: Обученный агент PPO.
        env: Среда для тестирования.
        n_episodes: Количество эпизодов для тестирования.

    Returns:
    ---------------
        None
    """
    for episode in range(n_episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0

        # Выполнение эпизода до завершения
        while not done:
            # Получение действия от агента
            action, _ = ppo.get_action(obs)

            # Выполнение шага в среде
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            # Накопление награды
            total_reward += reward

            # Рендеринг среды
            env.render()

        # Вывод общей награды за эпизод
        print(f"Эпизод {episode + 1}, Общая награда: {total_reward}")

    # Закрытие среды после тестирования
    env.close()

In [18]:
test_agent(ppo, env)

Эпизод 1, Общая награда: -1515.1427226525689
Эпизод 2, Общая награда: -1309.9893423038995
Эпизод 3, Общая награда: -1082.6582799700243
Эпизод 4, Общая награда: -1239.5453566172735
Эпизод 5, Общая награда: -1110.7083583066783
Эпизод 6, Общая награда: -916.0654989379681
Эпизод 7, Общая награда: -1047.2785855014663
Эпизод 8, Общая награда: -1306.5437231937635
Эпизод 9, Общая награда: -1161.8973842454343
Эпизод 10, Общая награда: -859.6355777379283
